In [38]:
import pandas as pd
import numpy as np

In [45]:
def get_SKU (ASIN_SKU:dict, ASIN:str):
    if ASIN in ASIN_SKU:
        return ASIN_SKU[ASIN]

In [72]:
def get_number(SKU:str):
    if SKU is not None:
        if len(SKU) < 12:
            return SKU[0:len(SKU)-3]
        else:
            return 0
    

In [80]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime('%d-%m-%Y')

def get_package_for_Ovolt(path_ana,path_inv,country_code:str,package_size:int):
    
    print("ładuje pliki")
    all_checked = pd.read_excel(path_ana)
    out = pd.read_csv(path_inv, sep='\t')
    print("pliki załadowane")

    temp = pd.DataFrame(out[['seller-sku','asin1']])
    
    asin_dict = {}
    for index, row in temp.iterrows():
        asin_dict[row['asin1']] = row['seller-sku']
    
    all_checked['SKU'] = [get_SKU(asin_dict, row) for row in all_checked['ASIN']]
    all_checked['Number'] = [get_number(row) for row in all_checked['SKU']]
    
    all_checked = all_checked.drop_duplicates(subset=['ASIN', 'Product ID'], keep='first')
    all_checked = all_checked.sort_values(by='Sales Rank', ascending=True)

    profit = all_checked[((all_checked['Profit']>0) | (all_checked['Profit'].isna())) & all_checked['Number'].isna()]

    all_checked.reset_index(inplace=True)
    profit.reset_index(inplace=True)

    max_value = pd.Series(all_checked['Number'])
    max_value = max_value.dropna()
    max_value = max_value.astype('int')
    start = max_value.max() 
    increment = 1
    print('numeruje komórki')
    profit['Number'] = start + increment + profit.index
    print('komórki ponumerowane')

    profit['Number'] = profit['Number'].astype('int')
    
    def generate_sku(row):
        return str(row['Number']) + f"_{country_code}"

    profit['SKU'] = profit.apply(generate_sku, axis=1)

    to_amazon = profit.head(package_size)
    to_amazon = to_amazon[['SKU', 'ASIN', 'cena']]
    to_amazon['cena'] = [row*2 for row in to_amazon['cena']]
    print('zapisuje plik do xlsx')
    to_amazon.to_excel(f"F:\\Ovolt\\{country_code}_{str(package_size)}_{str(formatted_date)}.xlsx", index=False)
    print(f'plik zapisany: {country_code}_{str(package_size)}_{str(formatted_date)}.xlsx')
    

In [81]:
get_package_for_Ovolt('G:\\Mój dysk\\Projekt\\P531 Ovolt\\Nowy folder (2)\\de-final.xlsx','F:\\Ovolt\\All+Listings+Report+08-25-2023.txt','FR',30000)

ładuje pliki
pliki załadowane
numeruje komórki
komórki ponumerowane


C:\Users\48575\AppData\Local\Temp\ipykernel_228\4120099678.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit['Number'] = start + increment + profit.index
C:\Users\48575\AppData\Local\Temp\ipykernel_228\4120099678.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit['Number'] = profit['Number'].astype('int')
C:\Users\48575\AppData\Local\Temp\ipykernel_228\4120099678.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

zapisuje plik do xlsx
plik zapisany: FR_30000_25-08-2023.xlsx
